In [84]:
import sys 
import os
sys.path.append('../')
import numpy as np
import torch
import skimage 
import matplotlib.pyplot as plt


In [ ]:

os.environ['nnUNet_raw_data_base'] = '/home/paulw/projects/TRUSnet-1/data/nnUNet_raw_data_base'
os.environ['nnUNet_preprocessed'] = '/home/paulw/projects/TRUSnet-1/data/nnUNet_preprocessed'
os.environ['RESULTS_FOLDER'] = '/home/paulw/projects/TRUSnet-1/logs/nnUnet'
NNUNET_RAW_DATA = os.environ['nnUNet_raw_data_base']
NNUNET_PREPROCESSED = os.environ['nnUNet_preprocessed']
RESULTS_FOLDER = os.environ['RESULTS_FOLDER']

TASK_ID = 501 
TASKNAME = 'Prostate'

TASK_RAW_DATA = os.path.join(
    NNUNET_RAW_DATA,
    'nnUNet_raw_data', 
    f'Task{TASK_ID}_{TASKNAME}'
)

TRAINING_RAW_DATA = os.path.join(
    TASK_RAW_DATA,
    'imagesTr'
)

TRAINING_RAW_LABELS = os.path.join(
    TASK_RAW_DATA,
    'labelsTr'
)

TESTING_RAW_DATA = os.path.join(
    TASK_RAW_DATA,
    'imagesTs'
)

OUT_SHAPE = (1012, 1012)

assert os.path.exists(TASK_RAW_DATA)

In [61]:
# get the correct cores

from src.data.exact.splits import Splits, HasProstateMaskFilter
splits = Splits(
    cohort_specifier=['UVA600', 'CRCEO428'],
    merge_test_centers=True,
    merge_val_centers=True, 
    merge_train_centers=True, 
    undersample_benign_eval=False, 
    undersample_benign_train=False,
)
splits.apply_filters(HasProstateMaskFilter())
train_cores = splits.get_train() + splits.get_val()
test_cores = splits.get_test()

In [62]:
print(f'{len(test_cores)=}')
print(f'{len(train_cores)=}')

len(test_cores)=171
len(train_cores)=742


In [63]:
import nnunet
from nnunet.utilities.file_conversions import convert_2d_image_to_nifti

In [89]:
# sample the saving functions 
import numpy as np 
import skimage
from src.data.exact.core import Core
core = Core.sample_core()
bmode = core.bmode
prostate_mask = core.prostate_mask


bmode = skimage.transform.resize(bmode, OUT_SHAPE, preserve_range=True, anti_aliasing=True)
bmode = (bmode - bmode.min()) / (bmode.max() - bmode.min()) 
bmode = (bmode * 255).astype(np.uint8)
skimage.io.imsave('bmode.png', bmode)


prostate_mask = skimage.transform.resize(prostate_mask, OUT_SHAPE, order=0, preserve_range=True, anti_aliasing=False)
prostate_mask = prostate_mask.astype(np.uint8)

skimage.io.imsave('mask.png', prostate_mask)

/home/paulw/projects/TRUSnet-1/notebooks/../src/data/exact/core.py:226: UserWarning: You appear to be calling Core constructor manually. It is preferable to call `Core.create_core(specifier)`
                    which will cache core instances and not double memory requirements for core objects!
                
  warn(
/tmp/ipykernel_1891/1883741500.py:19: UserWarning: mask.png is a low contrast image
  skimage.io.imsave('mask.png', prostate_mask)


In [112]:
from tqdm.notebook import tqdm

for i, core_specifier in enumerate(tqdm(train_cores)): 
    core = Core.create_core(core_specifier)
    bmode = core.bmode
    prostate_mask = core.prostate_mask
    bmode = skimage.transform.resize(bmode, OUT_SHAPE, preserve_range=True, anti_aliasing=True)
    bmode = (bmode - bmode.min()) / (bmode.max() - bmode.min()) 
    bmode = (bmode * 255).astype(np.uint8)
    skimage.io.imsave('bmode.png', bmode)
    convert_2d_image_to_nifti(
        'bmode.png', 
        os.path.join(TRAINING_RAW_DATA, f'Exact_{str.zfill(str(i), 3)}')
    )
    prostate_mask = skimage.transform.resize(prostate_mask, OUT_SHAPE, order=0, preserve_range=True, anti_aliasing=False)
    prostate_mask = prostate_mask.astype(np.uint8)
    skimage.io.imsave('mask.png', prostate_mask)
    convert_2d_image_to_nifti(
        'mask.png', 
        os.path.join(TRAINING_RAW_LABELS, f'Exact_{str.zfill(str(i), 3)}'),
        is_seg=True
    )

  0%|          | 0/742 [00:00<?, ?it/s]

/tmp/ipykernel_1891/3557572032.py:17: UserWarning: mask.png is a low contrast image
  skimage.io.imsave('mask.png', prostate_mask)


In [109]:
for i, core_specifier in enumerate(tqdm(test_cores)): 
    core = Core.create_core(core_specifier)
    bmode = core.bmode
    prostate_mask = core.prostate_mask
    bmode = skimage.transform.resize(bmode, OUT_SHAPE, preserve_range=True, anti_aliasing=True)
    bmode = (bmode - bmode.min()) / (bmode.max() - bmode.min()) 
    bmode = (bmode * 255).astype(np.uint8)
    skimage.io.imsave('bmode.png', bmode)
    convert_2d_image_to_nifti(
        'bmode.png', 
        os.path.join(TESTING_RAW_DATA, f'Exact_{str.zfill(str(i), 3)}')
    )

  0%|          | 0/171 [00:00<?, ?it/s]

In [111]:
from nnunet.dataset_conversion.utils import generate_dataset_json

generate_dataset_json(
    os.path.join(TASK_RAW_DATA, 'dataset.json'),
    os.path.join(TASK_RAW_DATA, 'imagesTr'),
    os.path.join(TASK_RAW_DATA, 'imagesTs'),
    ('ultrasound',),
    {0: 'background', 1: 'prostate'},
    "Exact_ultrasound", 
)